In [1]:
import lightgbm as lgb
from sklearn import metrics
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('../data/features.csv',dtype={'tipodepropiedad':'category','ciudad':'category','provincia':'category'})
df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df["mes"] = df['fecha'].dt.month
df["dia"] = df['fecha'].dt.day

In [4]:
df = df.drop(columns=["direccion","id","descripcion","titulo"])
#"lat","lng",

In [5]:
def imputar(df):
    imp = SimpleImputer()
    cols = df._get_numeric_data().columns
    df_result = df.copy()
    for x in cols:
        df_result[x]=imp.fit_transform(df[[x]])
    return df_result

In [6]:
df1 = imputar(df)

oneHot = ['tipodepropiedad','provincia']
label = ['ciudad']

cat_features = ['ciudad','tipodepropiedad','provincia']

df1 = df1.dropna(subset=cat_features)

In [7]:
def predecir(model, train_features, train_labels, test_features, test_labels):
    predict = model.score(test_features,test_labels)
    print('Entrenamiento: {:0.4f}%'.format(model.score(train_features, train_labels)*100))
    print('Testeo: {:0.4f}%.'.format(predict*100))
    
    return predict

In [30]:
data_cols = df1._get_numeric_data().columns
baseline_data = df1[data_cols]

encoder = ce.OneHotEncoder()

encoded = encoder.fit_transform(df1[oneHot])
data = baseline_data.join(encoded)

encoder = LabelEncoder()

encoded = encoder.fit_transform(df1[label])
data["ciudad"] = encoded

x = data[data.columns.drop("precio")]
y = data['precio']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

print(f"Original shapes: X={x.shape} y={y.shape}")
print(f"Train shapes: X={x_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={x_test.shape}  y={y_test.shape}")

/home/peter/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Original shapes: X=(239954, 87) y=(239954,)
Train shapes: X=(191963, 87) y=(191963,)
Test  shapes: X=(47991, 87)  y=(47991,)


In [31]:
xgb_model = XGBRegressor(n_estimatos=100,max_depth=10,n_jobs=-1,objective="reg:squarederror")


In [ ]:
xgb_model.fit(x_train, y_train)

In [ ]:
predecir(xgb_model,x_train,y_train,x_test,y_test)

In [ ]:
plt.rcParams["figure.figsize"] = [5, 15]

importancia = xgb_model.feature_importances_
ax = plt.barh(x.columns, xgb_model.feature_importances_)